In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt
import os.path

In [ ]:
%pip install numpy pandas scipy scikit-learn matplotlib

In [ ]:
if not os.path.isfile(path='book-crossings.zip'):
	!wget 'https://cdn.freecodecamp.org/project-data/books/book-crossings.zip'
	!unzip 'book-crossings.zip'

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# get users who have at least 200 reviews
reviews_per_user = pd.DataFrame(df_ratings['user'].value_counts())
valid_users = reviews_per_user[reviews_per_user['count'] >= 200].index

# get books that have at least 100 reviews
reviews_per_isbn = pd.DataFrame(df_ratings['isbn'].value_counts())
valid_isbns = reviews_per_isbn[reviews_per_isbn['count'] >= 100].index

# filter out reviews for users without enough reviews,
# then filter out reviews for books without enough reviews
df_reviews_filtered = df_ratings[df_ratings['user'].isin(valid_users)]
df_reviews_filtered = df_reviews_filtered[df_reviews_filtered['isbn'].isin(valid_isbns)]

# join books df on filtered reviews to add author and book title info
df_merged = pd.merge(right=df_reviews_filtered, left=df_books, on='isbn')
# df_merged

# encode categorical data to integers
# df_encoder = OrdinalEncoder()
# df_encoder.fit(df_merged)

# df_encoded = df_encoder.transform(df_merged)
# df_encoded

# print(pd.DataFrame(df_merged.iloc[0]).T)
# df_encoded = df_encoder.transform(pd.DataFrame(df_merged.iloc[0]).T)
# print(df_encoder.categories_[0][int(df_encoded[0][0])])

#pivot table to show relation between a book and a users rating
df_pivot = df_merged.pivot_table(index='title', columns='user', values='rating').fillna(0)
display(df_pivot)



In [ ]:
# create model
model = NearestNeighbors(n_neighbors=6, metric='cosine').fit(df_pivot)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
	distances, indices = model.kneighbors(pd.DataFrame(df_pivot.loc[book]).T)

	recommended_books = []
	for i in range(len(indices.flatten())):
		if i != 0:
			title = df_pivot.iloc[indices.flatten()[i]].name
			recommended_books.append([title, distances.flatten()[i]])

	recommended_books.reverse()

	recommended_books = [book, recommended_books]

	return recommended_books

In [ ]:
get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()